In [6]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import plotly.graph_objects as go
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps
import plotly.express as px
import base64

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from Animal_Shelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name
username = "aacuser"
password = "Dino"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read_all({}), columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                              'age_upon_outcome_in_weeks'])
         


#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')

#FIX ME Add in Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
#html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()))),
    html.Center(html.H1('Rico Applewhite:CS-340')),
    html.Hr(),
    html.H4("Potential Canidates for Search and Rescue Training"),
    html.P("Select training type:"),
    html.Div([
        dcc.RadioItems(
        id="radio-id",
        options=[
            {'label':'Water Rescue','value':'WR'},
            {'label':'Mountain or Wilderness Rescue','value':'MWR'} ,
            {'label':'Disaster or Individual Tracking','value':'DIT'},
            {'label':'Reset','value':'Default'}],
        value='Reset'
        )
    ]),
    html.Hr(),
    html.Div([
    dt.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        derived_viewport_data= [{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=True,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current=0,
        page_size=6,
#FIXME: Set up the features for your interactive data table to make it user-friendly for your client
#If you completed the Module Six Assignment, you can copy in the code you created here 
        )]),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.H4('Interactive chart for animal status'),         
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('radio-id', 'value')]
)
def update_dashboard(radio_value):
   
    if radio_value == "WR":
        df = pd.DataFrame.from_records(shelter.read_all({ '$and':[{'$or':[
        {'breed':'Labrador Retriever Mix'},{'breed':'Chesapeake Bay Retriever'},{'breed':'Newfoundland'}]},
        {'sex_upon_outcome': 'Intact Female'},
        {'$and':[{'age_upon_outcome_in_weeks': {'$gte': 26}},
        {'age_upon_outcome_in_weeks': {'$lte': 156}},
       ]}]}), columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                              'age_upon_outcome_in_weeks'])
    
    elif radio_value == "MWR":
        df = pd.DataFrame.from_records(shelter.read_all({ '$and':[{'$or':[
        {'breed':'German Shepherd'},{'breed':'Alaskan Malamute'},{'breed':'Old English Sheepdog'},
        {'breed':'Siberian Husky'},{'breed':'Rottweiler'}]},
        {'sex_upon_outcome': 'Intact Male'},
        {'$and':[{'age_upon_outcome_in_weeks': {'$gte': 26}},
        {'age_upon_outcome_in_weeks': {'$lte': 156}},
       ]}]}), columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                              'age_upon_outcome_in_weeks'])
        
    elif radio_value == "DIT":
        df = pd.DataFrame.from_records(shelter.read_all({ '$and':[{'$or':[
        {'breed':'Doberman Pinscher'},{'breed':'German Shepherd'},{'breed':'Golden Retriever'},
        {'breed':'Bloodhound'},{'breed':'Rottweiler'}]},
        {'sex_upon_outcome': 'Intact Male'},
        {'$and':[{'age_upon_outcome_in_weeks': {'$gte': 20}},
        {'age_upon_outcome_in_weeks': {'$lte': 300},
       }]}]}), columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                              'age_upon_outcome_in_weeks'])
        
    elif radio_value == "Default":
        df = pd.DataFrame.from_records(shelter.read_all({}), columns=['id', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed', 'color',
                                              'date_of_birth', 'datetime', 'monthyear', 'name', 'outcome_subtype',
                                              'outcome_type', 'sex_upon_outcome', 'location_lat', 'location_long',
                                              'age_upon_outcome_in_weeks'])

        

        
    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
    data=df.to_dict('records')
    return (data,columns)        
    
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData).size
    return [
        dcc.Graph(            
            figure =px.bar(df, x='animal_id', y='outcome_type')
)    
    ]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', 'derived_viewport_data')])
#    Input('datatable_id', 'selected_columns')])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    selected_columns=4
    # Austin TX is at [30.75,-97.48]
    
    return [
        dl.Map(
            style={"width": "1000px", "height": "500px"},
            center=[
                float(dff.iloc[selected_columns, 13]),
                float(dff.iloc[selected_columns, 14]),
            ],  # [30.75, -97.48],
            zoom=10,
            children=[
                dl.TileLayer(id=f"base_layer_id"),  # _{index}
                # Marker with tool tip and popup
                dl.Marker(
                    position=[
                        float(dff.iloc[selected_columns, 13]),
                        float(dff.iloc[selected_columns, 14]),
                    ],
                    children=[
                        dl.Tooltip(dff.iloc[selected_columns, 4]),  # dff.iloc[0, 4]),
                        dl.Popup(
                            [html.H2(dff.iloc[selected_columns, 2]),html.H3("Animal Name"), html.P(dff.iloc[selected_columns, 9])]
                        ),  # dff.iloc[1, 9])]),
                ])
            ])
        ]
    
#FIXME: Add in the code for your geolocation chart
#If you completed the Module Six Assignment, you can copy in the code you created here.



app